### Use case

Celem ćwiczenia jest przewidzenie jaki rezultat otrzymają uczniowie z testu matematycznego znając inne dane na ich temat takie jak:
* płeć
* rase/pochodzenię
* wykształcenie rodziców
* wyniki z innych testów
* poziom majętności rodziców na podstawie przysługujących zniżek na lunch
* ukończenie testu przygotowawczego

Link do dataset'u: https://www.kaggle.com/spscientist/students-performance-in-exams  
Przewidywana kolumna: **math result**  
Kolumny brane pod uwagę: **wszystkie**

### Kroki do wykonania eksperymentu

1. Stworzenie resource group'y
2. Stworzenie resource'a Azure Machine Learning
3. Zalogowanie się do portalu [ml.azure.com] i stworzenie środowiska w naszej resource group'ie do przeprowadzaenia eksperymentu
4. Pobranie datasetu z kaggle'a i załadowanie go do zestawu danych na [ml.azure.com] w formie tabelarycznej z nagłówkiem określającym nazwy kolumn i nazwą **students-performance**
5. Stworzenie dwóch środowisk obliczeniowych (najlepiej najtańszych). Jedno będzie służyło nam do uruchomienia jupytera i eksperymentu i może mieć dowolną nazwę. Na drugim będzie przeprowadzony eksperyment i powinno mieć nazwę **regression-cluster**.
6. Wrzucenie tego notebook'a do plików w środowisku i uruchomienie go

### Sprawdzamy czy workspace jest gotowy do użycia

In [7]:
import azureml.core
from azureml.core import Workspace, Dataset

print("SDK version:", azureml.core.VERSION)
ws = Workspace.from_config()

SDK version: 1.19.0


### Wczytujemy dane do dataframe'a w pandasie

In [8]:
students_dataset = ws.datasets['students-performance']
students_df = students_dataset.to_pandas_dataframe()
students_df.head(5)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


### Dzielimy zbiór na treningowy i testowy

In [9]:
train_dataset, test_dataset = students_dataset.random_split(0.9, seed=1)

train_dataset_df = train_dataset.to_pandas_dataframe()
test_dataset_df = test_dataset.to_pandas_dataframe()

train_dataset_df.head(5)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


### Sprawdzamy dostępne środowiska obliczeniowe

In [10]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

ComputeTarget.list(ws)

[{
   "id": "/subscriptions/77f26125-4a7b-48ba-a510-6e1c36506c3b/resourceGroups/auto-ML/providers/Microsoft.MachineLearningServices/workspaces/ml-resource/computes/covid-cluster",
   "name": "covid-cluster",
   "location": "westeurope",
   "tags": null,
   "properties": {
     "description": null,
     "computeType": "ComputeInstance",
     "computeLocation": "westeurope",
     "resourceId": null,
     "provisioningErrors": null,
     "provisioningState": "Succeeded",
     "properties": {
       "vmSize": "STANDARD_DS2_V2",
       "applications": [
         {
           "displayName": "Jupyter",
           "endpointUri": "https://covid-cluster.westeurope.instances.azureml.ms"
         },
         {
           "displayName": "Jupyter Lab",
           "endpointUri": "https://covid-cluster.westeurope.instances.azureml.ms/lab"
         },
         {
           "displayName": "RStudio",
           "endpointUri": "https://covid-cluster-8787.westeurope.instances.azureml.ms"
         }
       

### Podłączamy się do klastra na którym będziemy wykonywali obliczenia

In [13]:
amlcompute_cluster_name = "regression-cluster"
cts = ws.compute_targets
aml_remote_compute = cts[amlcompute_cluster_name]
aml_remote_compute.wait_for_completion(show_output = True)
aml_remote_compute.get_status().serialize()


Running


{'errors': [],
 'creationTime': '2020-12-26T14:03:56.522441+00:00',
 'createdBy': {'userObjectId': '1d574843-1d03-4c9c-bb14-60d950cee417',
  'userTenantId': 'dea32f1c-cf5d-4d7d-a4d0-3cf7088348c7',
  'userName': None},
 'modifiedTime': '2020-12-26T14:06:12.708420+00:00',
 'state': 'Running',
 'vmSize': 'STANDARD_DS2_V2'}

### Sprawdzamy metryki dla problemu regresji

In [14]:
from azureml.train import automl
automl.utilities.get_primary_metrics('regression')

['normalized_mean_absolute_error',
 'r2_score',
 'normalized_root_mean_squared_error',
 'spearman_correlation']

### Definiujemy eksperyment
Błąd eksperymentu będzie mierzony metodą znormalizowanego średniego bewzględnego błedu.

In [15]:
import logging
import os
from azureml.train.automl import AutoMLConfig

project_folder = './automl'
os.makedirs(project_folder, exist_ok=True)

automl_config = AutoMLConfig(compute_target=aml_remote_compute,
                             task='regression',
                             primary_metric='normalized_mean_absolute_error',
                             experiment_timeout_minutes=15,                            
                             training_data=train_dataset,
                             label_column_name="math score",
                             n_cross_validations=5,                                                   
                             enable_early_stopping=True,
                             featurization='auto',
                             debug_log='automated_ml_errors.log',
                             verbosity=logging.INFO,
                             path=project_folder)

### Uruchamiamy eksperyment

Czas trwania eksperymentu to około 12 min. Po przeprowadzeniu eksperymentu wyświetlą się nam wyniki dla różnych modeli.

In [16]:
from azureml.core import Experiment
from datetime import datetime

now = datetime.now()
time_string = now.strftime("%m-%d-%Y-%H")
experiment_name = "classif-automl-remote-{0}".format(time_string)
print(experiment_name)

experiment = Experiment(workspace=ws, name=experiment_name)

import time
start_time = time.time()
            
run = experiment.submit(automl_config, show_output=True)

print('Manual run timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (time.time() - start_time))

classif-automl-remote-12-26-2020-14
Running on remote.
No run_configuration provided, running on regression-cluster with default configuration
Running on remote compute: regression-cluster
Parent Run ID: AutoML_4a7b380a-e712-460a-9ed8-10956d425534

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************

### Wymagany czas na aksperyment

In [18]:
import time
import datetime as dt

run_details = run.get_details()

# Like: 2020-01-12T23:11:56.292703Z
end_time_utc_str = run_details['endTimeUtc'].split(".")[0]
start_time_utc_str = run_details['startTimeUtc'].split(".")[0]
timestamp_end = time.mktime(datetime.strptime(end_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())
timestamp_start = time.mktime(datetime.strptime(start_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())

parent_run_time = timestamp_end - timestamp_start
print('Run Timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (parent_run_time))

Run Timing: --- 1363.0 seconds needed for running the whole Remote AutoML Experiment ---


### Najlepszy model
W moim przypadku najlepszym modelem okazał się model **VotingEnsemble**

In [19]:
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: classif-automl-remote-12-26-2020-14,
Id: AutoML_4a7b380a-e712-460a-9ed8-10956d425534_16,
Type: azureml.scriptrun,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                          

### Wykonanie predykcji na zbiorze testowym najlepszym modelem

In [21]:
import pandas as pd

if 'math score' in test_dataset_df.columns:
    y_test_df = test_dataset_df.pop('math score')

x_test_df = test_dataset_df
y_predictions = fitted_model.predict(x_test_df)

### Sprawdzamy jaki otrzymaliśmy średni błąd absolutny na zbiorze testowym

In [27]:
from sklearn.metrics import mean_absolute_error

print('Mean absolute error:')
mean_absolute_error(y_test_df, y_predictions)

Mean absolute error:


4.98317215598992